In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
df_shows = pd.read_csv('../Data/raw/movies/HuluRaw.csv')

In [3]:
df_shows.dropna(axis=1, how='any', inplace=True)
df_shows.head(2)

,show/id,show/cache_time,show/canonical_name,show/clips_count,show/description,show/episodes_count,show/feature_films_count,show/film_clips_count,show/genre,show/genres,...,show/show_rollups/showtime/videos_count,show/show_rollups/showtime/latest_video_added_at,show/show_rollups/showtime/html5_clips_count,show/show_rollups/showtime/html5_episodes_count,show/show_rollups/showtime/html5_games_count,show/show_rollups/showtime/html5_feature_films_count,show/show_rollups/showtime/html5_film_clips_count,show/show_rollups/showtime/html5_film_trailers_count,show/show_rollups/showtime/html5_videos_count,show/show_rollups/showtime/non_rating_videos_count
0,54,2017-08-10T14:53:04+00:00,family-guy,288,The adventures of an endearingly ignorant dad ...,288,0,0,Animation and Cartoons,Animation and Cartoons~Primetime Animation|Tee...,...,576,2017-07-22T21:15:30Z,288,0,0,0,0,0,288,0
1,6979,2017-08-10T14:14:33+00:00,south-park,3874,"Underpants-stealing gnomes, a talking Christma...",274,0,0,Comedy,Comedy|Animation and Cartoons~Primetime Animation,...,4148,2016-12-08T06:32:22Z,3874,274,0,0,0,0,4148,0


In [4]:
df_shows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 156 entries, show/id to show/show_rollups/showtime/non_rating_videos_count
dtypes: bool(6), float64(1), int64(122), object(27)
memory usage: 1.2+ MB


We must inspect some of this columns to know which are useful to analize. We select the following:

In [5]:
columns_interest = ['show/id', 'show/canonical_name', 'show/clips_count', 
                    'show/description', 'show/episodes_count', 'show/feature_films_count', 
                    'show/genre', 'show/genres', 'show/is_movie',
                    'show/is_showtime_only', 'show/rating', 'show/series_id', 'show/company/name']

In [6]:
df_shows_ = df_shows.loc[:, columns_interest]
df_shows_.head()

,show/id,show/canonical_name,show/clips_count,show/description,show/episodes_count,show/feature_films_count,show/genre,show/genres,show/is_movie,show/is_showtime_only,show/rating,show/series_id,show/company/name
0,54,family-guy,288,The adventures of an endearingly ignorant dad ...,288,0,Animation and Cartoons,Animation and Cartoons~Primetime Animation|Tee...,False,False,4.353707,11730,FOX
1,6979,south-park,3874,"Underpants-stealing gnomes, a talking Christma...",274,0,Comedy,Comedy|Animation and Cartoons~Primetime Animation,False,False,4.363032,50003814,Comedy Central
2,53,american-dad,121,This screwball family full of radically differ...,172,0,Comedy,Comedy~Sitcoms|Animation and Cartoons~Primetim...,False,False,4.114007,379,Fox Television Classics
3,389,law-and-order-special-victims-unit,176,This hard-hitting and emotional series from NB...,406,0,Drama,Drama~Crime and Courtroom Drama|Drama~Mystery,False,False,4.622856,260,NBC
4,1603,naruto-shippuden,16,The Village Hidden in the Leaves is home to th...,597,0,Anime,Anime~Anime - Action Adventure|Action and Adve...,False,False,4.400626,10960,Viz Media


In [7]:
try:
    assert len(df_shows_['show/id']) == len(df_shows_['show/id'].unique())
except:
    df_shows_.drop_duplicates(subset='show/id', inplace=True)

assert len(df_shows_['show/id']) == len(df_shows_['show/id'].unique()) 

In [8]:
genre_list = []
ind = []
for idx, genre in zip(df_shows_['show/id'], df_shows_['show/genres']):
    elems = genre.split('|')
    for elem in elems:
        genre_list.append(elem)
        ind.append(idx)

In [9]:
df_genre = pd.DataFrame({'id_show': ind, 'genres':genre_list})
df_genre = df_genre.pivot(index='id_show', columns='genres', values='genres')  

In [10]:
df_genre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 10 to 29524
Data columns (total 42 columns):
Action and Adventure                                3 non-null object
Action and Adventure~Espionage                      1 non-null object
Action and Adventure~Military and War               1 non-null object
Animation and Cartoons                              9 non-null object
Animation and Cartoons~Primetime Animation          8 non-null object
Animation and Cartoons~Saturday Morning Cartoons    2 non-null object
Anime                                               3 non-null object
Anime~Anime - Action Adventure                      2 non-null object
Anime~Anime - Supernatural and Horror               1 non-null object
Anime~Anime - Tournament                            1 non-null object
Classics                                            1 non-null object
Comedy                                              19 non-null object
Comedy~Late Night Comedy                           

In [11]:
df_genre_transform = pd.get_dummies(df_genre)
df_genre_transform.head()

,Action and Adventure_Action and Adventure,Action and Adventure~Espionage_Action and Adventure~Espionage,Action and Adventure~Military and War_Action and Adventure~Military and War,Animation and Cartoons_Animation and Cartoons,Animation and Cartoons~Primetime Animation_Animation and Cartoons~Primetime Animation,Animation and Cartoons~Saturday Morning Cartoons_Animation and Cartoons~Saturday Morning Cartoons,Anime_Anime,Anime~Anime - Action Adventure_Anime~Anime - Action Adventure,Anime~Anime - Supernatural and Horror_Anime~Anime - Supernatural and Horror,Anime~Anime - Tournament_Anime~Anime - Tournament,...,Music_Music,Music~Musicals_Music~Musicals,News and Information~Talk and Interview_News and Information~Talk and Interview,Reality and Game Shows_Reality and Game Shows,Reality and Game Shows~Competition_Reality and Game Shows~Competition,Reality and Game Shows~Real Life Drama_Reality and Game Shows~Real Life Drama,Science Fiction_Science Fiction,Science Fiction~Science Fiction - Thriller_Science Fiction~Science Fiction - Thriller,Teen_Teen,Videogames_Videogames
id_show,,,,,,,,,,,,,,,,,,,,,
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


It's time to group some genres categories. First, we inspect how many shows there are in every category. Some of them have less of 5 shows or even just 1. In that case, we group by similarity of genre names. For instances, `Action and Adventure`, `Action and Adventure~Espionage`, `Action and Adventure~Military and War` could be one category and so on.

Category `Animation`:

In [12]:
df_genre_transform['Animation'] = \
df_genre_transform['Animation and Cartoons_Animation and Cartoons'] + \
df_genre_transform['Animation and Cartoons~Primetime Animation_Animation and Cartoons~Primetime Animation'] + \
df_genre_transform['Animation and Cartoons~Saturday Morning Cartoons_Animation and Cartoons~Saturday Morning Cartoons']

df_genre_transform.drop(columns=['Animation and Cartoons_Animation and Cartoons',
                                'Animation and Cartoons~Primetime Animation_Animation and Cartoons~Primetime Animation',
                                'Animation and Cartoons~Saturday Morning Cartoons_Animation and Cartoons~Saturday Morning Cartoons'],
                        axis=1, inplace=True)


Cateogory `Anime`:

In [13]:
df_genre_transform['Anime'] = \
df_genre_transform['Anime~Anime - Action Adventure_Anime~Anime - Action Adventure'] + \
df_genre_transform['Anime_Anime'] + \
df_genre_transform['Anime~Anime - Supernatural and Horror_Anime~Anime - Supernatural and Horror'] + \
df_genre_transform['Anime~Anime - Tournament_Anime~Anime - Tournament']

df_genre_transform.drop(columns=['Anime~Anime - Action Adventure_Anime~Anime - Action Adventure',
                                'Anime_Anime',
                                'Anime~Anime - Supernatural and Horror_Anime~Anime - Supernatural and Horror',
                                'Anime~Anime - Tournament_Anime~Anime - Tournament'],
                        axis=1, inplace=True)

Category `Action`:

In [14]:
df_genre_transform['Action'] = \
df_genre_transform['Action and Adventure_Action and Adventure'] + \
df_genre_transform['Action and Adventure~Espionage_Action and Adventure~Espionage'] + \
df_genre_transform['Action and Adventure~Military and War_Action and Adventure~Military and War']

df_genre_transform.drop(columns=['Action and Adventure_Action and Adventure',
                                'Action and Adventure~Espionage_Action and Adventure~Espionage',
                                'Action and Adventure~Military and War_Action and Adventure~Military and War'],
                        axis=1, inplace=True)

Category `Comedy`:

In [16]:
df_genre_transform['Comedy'] = df_genre_transform['Comedy_Comedy'] + \
df_genre_transform['Comedy~Late Night Comedy_Comedy~Late Night Comedy'] + \
df_genre_transform['Comedy~Romantic Comedy_Comedy~Romantic Comedy'] + \
df_genre_transform['Comedy~Sitcoms_Comedy~Sitcoms'] + df_genre_transform['Comedy~Sketch Comedy_Comedy~Sketch Comedy']

df_genre_transform.drop(columns=['Comedy_Comedy',
                                'Comedy~Late Night Comedy_Comedy~Late Night Comedy',
                                'Comedy~Romantic Comedy_Comedy~Romantic Comedy',
                                'Comedy~Sitcoms_Comedy~Sitcoms', 'Comedy~Sketch Comedy_Comedy~Sketch Comedy'],
                        axis=1, inplace=True)

Category `Food`:

In [17]:
df_genre_transform['Food'] = \
df_genre_transform['Food_Food'] + df_genre_transform['Food~Cooking Competitions_Food~Cooking Competitions']  

df_genre_transform.drop(columns=['Food_Food',
                                'Food~Cooking Competitions_Food~Cooking Competitions'],
                        axis=1, inplace=True)

Category `Reality`:

In [18]:
df_genre_transform['Reality'] = \
df_genre_transform['Reality and Game Shows_Reality and Game Shows'] + \
df_genre_transform['Reality and Game Shows~Competition_Reality and Game Shows~Competition'] + \
df_genre_transform['Reality and Game Shows~Real Life Drama_Reality and Game Shows~Real Life Drama']

df_genre_transform.drop(columns=['Reality and Game Shows_Reality and Game Shows',
                                'Reality and Game Shows~Competition_Reality and Game Shows~Competition',
                                'Reality and Game Shows~Real Life Drama_Reality and Game Shows~Real Life Drama'],
                        axis=1, inplace=True)

In [19]:
df_genre_transform.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 10 to 29524
Data columns (total 28 columns):
Classics_Classics                                                                        109 non-null uint8
Drama_Drama                                                                              109 non-null uint8
Drama~Crime and Courtroom Drama_Drama~Crime and Courtroom Drama                          109 non-null uint8
Drama~Medical Drama_Drama~Medical Drama                                                  109 non-null uint8
Drama~Mystery_Drama~Mystery                                                              109 non-null uint8
Drama~Police Drama_Drama~Police Drama                                                    109 non-null uint8
Drama~Romance_Drama~Romance                                                              109 non-null uint8
Drama~Soap Operas_Drama~Soap Operas                                                      109 non-null uint8
Family_Family               

In [20]:
df_show_genres = df_shows_.merge(df_genre_transform, left_on='show/id', right_on='id_show')
df_show_genres.head(2)

,show/id,show/canonical_name,show/clips_count,show/description,show/episodes_count,show/feature_films_count,show/genre,show/genres,show/is_movie,show/is_showtime_only,...,Science Fiction_Science Fiction,Science Fiction~Science Fiction - Thriller_Science Fiction~Science Fiction - Thriller,Teen_Teen,Videogames_Videogames,Animation,Anime,Action,Comedy,Food,Reality
0,54,family-guy,288,The adventures of an endearingly ignorant dad ...,288,0,Animation and Cartoons,Animation and Cartoons~Primetime Animation|Tee...,False,False,...,0,0,1,0,1,0,0,1,0,0
1,6979,south-park,3874,"Underpants-stealing gnomes, a talking Christma...",274,0,Comedy,Comedy|Animation and Cartoons~Primetime Animation,False,False,...,0,0,0,0,1,0,0,1,0,0


In [21]:
#df_show_genres.to_csv('../Data/interim/movies/shows.csv')